# Improving Hydro Power Plants optimization using the HBV model

In this Notebook we first define the HBV model and the use it to get the inflow of water that goes to the hydro power plant that we are optimising.


## HBV 

We recall the different variables:    
- $a(t)$ the total water runoff on day $t$
- $a_0(t)$ the surface runoff
- $a_1(t)$ the interflow
- $a_2(t)$ the baseflow
- $K_0(t)$ the recession coefficient associated to the surface runflow
- $K_1(t)$ the recession coefficient associated to the inflow
- $a_2(t)$ the recession coefficient associated to the baseflow
- $T$ the last day of the model
- $\tilde{\theta}$ the temperature threshold for the processing of inputed precipitations
- $S_m(t)$ the snow melting rate
- $\theta(t)$ the temperature on day $t$
- $F$ the degree-day factor
- $FC$ the Field Capacity, ie the maximum storable soil moisture
- $SM(t)$ the soil moisture in L
- $P(t)$ the volume of rainfall water in L
- $\beta$ the shape coefficient for the moisture deficit
- $P_{eff}$ the effective precipitation that contributes to the surface runoff
- $\frac{SM(t)}{FC}$ the soil moisture deficit 
- $PE_a(t)$ the daily adjusted potential evapotranspiration
- $\bar{\theta}(t)$ the mean daily temperature
- $\theta_m$ the long term monthly mean temperature
- $PE_m$ the long term monthly mean potential evapotranspiration
- $E_a(t)$ the actual daily evapotranspiration
- $PWP$ the soil Permanent Wilting Point, ie the minimal soil moisture a plant requires not to wilt

In [7]:
import numpy as np
import datetime
import scipy.optimize as sco

In [8]:
def module_snowmelting(t, precipitations, temp, snow_pack, rainfall, F=1, temp_threshold=0):
    """
    Snow melting module
    
    Returns the amount of rainfall and snow accumulation.
    
    ------------------------------------------------------------
    
    Parameters:
    
    t, date - Running day
    precipitations, array - Expected precipitations for the running period
    temp, array - Expected temperatures for the running period
    snow_pack, array - Amount of snow accumulated
    F, float - Degree-day factor
    temp_threshold, float - Temperature threshold for snow to melt
    
    """
    
    if temp[t] > temp_threshold:
            # snow melts and precipitations are rainfalls
            rainfall[t] = precipitations[t] + F*(temp[t]-temp_threshold) * snow_pack[t-1]
            snow_pack[t] = snow_pack[t-1] *(1- F*(temp[t]-temp_threshold) )
    else:
        # there is no rainfall, precipitations contribute to the snow pack
        snow_pack[t] = snow_pack[t-1] + precipitations[t]
        rainfall[t] = 0

    return rainfall, snow_pack

def module_soilmoisture(t, rainfall, sm, effective_precipitation, FC=1, beta=1):
    """
    Soil moisture module
    
    Returns the updated soil moisture and the effective precipitation that contributes to the runoff.
        
    ------------------------------------------------------------
    
    Parameters:
    
    t, date - Running day
    rainfall, array - Amount of rain and melting snow that goes to the soil
    sm, array - Soil Moisture water amount
    FC, float - Field Capacity, ie maximum storage in the subsurface zone
    beta, positive integer - Shape Coefficient for the contribution to effective runoff
    
    """
    
    effective_precipitation[t] = rainfall[t] * (sm[t-1]/FC)**beta
    sm[t] = sm[t-1] + rainfall[t] * (1-(sm[t-1]/FC)**beta)
    
    return sm, effective_precipitation

def module_evapotranspiration(t,m, sm, ea, pea, temp,temp_m, pe_m, C=1, PWP=0):
    """
    Evapotranspiration module
    
    Returns the evapotranspiration and the soil moisture
    
    ------------------------------------------------------------
    
    Parameters:
    
    t, date, - Running day
    m, integer between 1 and 12 - Running month
    sm, array - Soil Moisture water amount
    temp, array - Expected temperatures for the running period
    temp_m, array - Long-term monthly mean temperatures
    pe_m, array -  Long-term mean monthly potential evapotranspiration 
    C, float - Model parameter
    PWP, float - Soil Permanent Wilting Poin
    
    """
    
    # compute the daily adjusted potential evapotranspiration
    pea[t] = (1+C * (temp[t]-temp_m[m])) * pe_m[m]
    
    # computes actual evapotranspiration
    if sm[t] < PWP:
        ea[t] = pea[t] * sm[t] / PWP
    else:
        ea[t] = pea[t]
    
    sm[t]-=ea[t]
    
    return ea, sm
def module_runoff(t, effective_precipitation, rl, ru, a0, a1, a2,perc, K=[1,1,1], L=200):
    """
    Runoff module
    
    Returns the three runoff discharges.
    
    ------------------------------------------------------------
    
    Parameters:
    
    t, date, - Running day
    L, float - Upper reservoir threshold 
    K, array - Recessions coefficients for each reservoir
    ru, array - Upper reservoir level
    rl, array - Lower reservoir level
    a0, array - Near surface runoff 
    a1, array - Inflow runoff 
    a2, array - Baseflow runoff 
    perc, float - Quantity that is percolated from the upper to the lower reservoir
    effective_precipitation, array - Effective contribution to the runoff
    
    """
    # inflow in reservoirs
    ru[t] = max(0, ru[t-1] + effective_precipitation[t] - perc)
    rl[t] = rl[t-1] + min(ru[t-1] + effective_precipitation[t], perc)
    
    # outflows
    a0[t] = K[0] * max(0, ru[t]-L)
    a1[t] = K[1] * (ru[t]-a0[t])
    a2[t] = K[2] * rl[t]
    
    # update reservoirs lvls
    ru[t] -=  (a0[t] + a1[t])
    rl[t] -=  a2[t]
    
    return rl, ru, a0, a1, a2

def total_runoff(t, a0, a1, a2):
    """
    Returns the total runoff discharge for one day.
    
    ------------------------------------------------------------
        
    Parameters:
    
    t, date, - Running day
    a0, array - Near surface runoff 
    a1, array - Inflow runoff 
    a2, array - Baseflow runoff 
    
    """
    
    return a0[t] + a1[t] + a2[t]

def HBV(days, precipitations, temp, pe_m, temp_m, init):
    
    """
    Returns the total runoff waters for each day.
        
    ------------------------------------------------------------
        
    Parameters:
    
    days, array of dates - Days of the running period
    precipitations, array - Expected precipitations for the running period
    temp, array - Expected temperatures for the running period
    temp_m, array - Long-term monthly mean temperatures
    pe_m, array -  Long-term mean monthly potential evapotranspiration
    sm0, float - Initial soil moisutre
    init, list - List of the initial values in the following order: sm0, rl0, ru0, snow_pack0 
    
    """
    
    # Parameters:
    F = 3
    temp_threshold = 0.0
    FC = 150
    PWP = 150
    C = 0.05
    beta = 1
    K = [0.2,0.1,0.05]
    L = 5
    Perc = 0.05
    
    T = len(days)
    
    # initialization
    sm = np.zeros(T)
    rainfall = np.zeros(T)
    rl = np.zeros(T)
    ru = np.zeros(T)
    total = np.zeros(T)
    a0 = np.zeros(T)
    a1 = np.zeros(T)
    a2 = np.zeros(T)
    effective_precipitation = np.zeros(T)
    snow_pack = np.zeros(T)
    ea = np.zeros(T)
    pea = np.zeros(T)
    
    sm0, rl0, ru0, snow_pack0 = init
    
    sm[0] = sm0
    rl[0] = rl0
    ru[0] = ru0
    snow_pack[0] = snow_pack0
    
    # loop over days
    for t in range(1, len(days)-1):
        
        m = days[t].month
        rainfall, snow_pack = module_snowmelting(t, precipitations, temp, snow_pack, rainfall, F, temp_threshold)
        sm, effective_precipitation =  module_soilmoisture(t, rainfall, sm, effective_precipitation, FC, beta)
        ea, sm = module_evapotranspiration(t,m, sm, temp, ea, pea, temp_m, pe_m, C, PWP)
        rl, ru, a0, a1, a2 = module_runoff(t, effective_precipitation, rl, ru, a0, a1, a2, Perc, K, L)
        total[t] = total_runoff(t, a0, a1, a2)
        
    return total

## Hydro Power Plant Optimisation

In [15]:
def objective(q):
    """
    Returns the payoff of selling q for price s 
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - the quantity of water flown in m^3
    s, array - prices in €/MWH
    h, float - head (ie altitude of the water fall) in m
    rho, float - density of water, = 1000kg/m^3
    g , float - gravitational constant, = 9.81m/s^2
    cf, float - conversion factor from Joule to MWH, =2.78 e-7
    
    """
    cf = 2.78e-7
    g = 9.81
    rho = 1000
    
    return -sum([x*y*h*rho*g*cf for x,y in zip(*[q,s])])

def cons_periodicflow_min(q):
    """
    Constraint on minimum flow during the period
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return q.sum()-qpmin

def cons_periodicflow_max(q):
    """
    Constraint on minimum flow during the period
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    return qpmax - q.sum()

def cons_daily_reservoir_max(q):
    """
    Constraint on daily maximum water level in reservoir
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return np.array([x0+np.sum(a[:t+1])-np.sum(q[:t+1]) +xmax[t] for t in range(0,T)])

def cons_daily_reservoir_min(q):
    """
    Constraint on daily minimum water level in reservoir
    
    ------------------------------------------------------------
    
    Parameters:
    
    q, array - quantity of water flown
    
    """
    
    return np.array([x0+np.sum(a[:t+1])-np.sum(q[:t+1]) -xmin[t] for t in range(0,T)])

con1 = {'type':'ineq','fun':cons_periodicflow_min}
con2 = {'type':'ineq','fun':cons_periodicflow_max}
con3 = {'type':'ineq','fun':cons_daily_reservoir_min}
con4 = {'type':'ineq','fun':cons_daily_reservoir_max}

cons=[con1,con2,con3,con4]


def HPP(q0,b,cons):
    """
    Optimizes the Hydro Power Plant (HPP)
    
    """
    sol = sco.minimize(objective,q0,bounds=b,constraints=cons)
    print("Payoff is :", -round(sol.fun,2))
    print("Optimal trajectory is :", [round(q,2) for q in sol.x])
    return sol

## Joint Use

Data we need:
- Hydro Plant:
    - $X$ Capacity
    - $qmax$ ($qmin$ can be assumed to be zero)
    - $Xmin$
- HBV:
    - precipitations: OK
    - temperatures: OK
    - mean monthly temperatures: OK
    - evapotranspiration
    - snow pack
    - starting soil moisture
    - starting reserves in upper and lower theoritical reservoirs

As an example we are taking the famous french "Grand'Maison dam". It has the following characteristics:
- X = 137 millions of $m^3$
- Capacity = 1820 MW
- head = 0.85 m
- flow capacity 216.3 $m^3/s$ ie 216.3 * 86400 = 18688320 $m^3/day$

In [16]:
# HBV parameters and outflow

sm0, rl0, ru0, snow_pack0 = 0,0,0,0
init = [sm0, rl0, ru0, snow_pack0]
days = [datetime.date.today() + datetime.timedelta(days=i) for i in range(10)]
pe_m = [10 for i in range(12)]
temp_m = [10 for i in range(12)]
precipitations = [5 for i in range(len(days)+1)]
temp = [5+i for i in range(len(days)+1)]

a_total = HBV(days, precipitations, temp, pe_m, temp_m, init)

In [22]:
# HPP parameters

a = a_total                                  # inflows in the reservoir (L)
T = len(a)                                   # number of days
q0 = np.zeros(T)                             # initial values for outflows (L)
s = np.array([10 for i in range(T)])         # spot price
x0 = 0.75 * 137*10e6                         # initial lvl of water in the reservoir ~ 75% full 
h = 0.85                                     # head

# constraints        
qpmax = 18688320*T                           # Max quantity of water flown out from the reservoir over the period, ie no cosntraint here
qpmin = 0                                    # Min quantity of water flown out from the reservoir over the period
xmin = [0.25* 137*10e6 for i in range(T)]    # Daily minimum lvl of water in the reservoir ~ 25% of full
x_max = 137*10e6                             # reservoir water capacity
xmax = [x_max for i in range(T)]

# bounds                 
qmin = [0 for i in range(T)]                 # daily minimum of water flown out from the reservoir
qmax = [18688320  for i in range(T)]         # daily minimum of water flown out from the reservoir
b = [(qmin[t],qmax[t]) for t in range(T)]    # bounds using the above

In [23]:
solution = HPP(q0,b,cons)

Payoff is : 4332145.07
Optimal trajectory is : [18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0, 18688320.0]
